# libraries

In [6]:
#import libraries
import pandas as pd
print('pandas: %s' % pd.__version__)

pd.options.display.max_columns = None
pd.set_option('display.max_rows', 150)
from keras.utils import Sequence

import numpy as np
print('geopandas: %s' % np.__version__)

# Tensorflow / Keras
import tensorflow as tf # used to access argmax function
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense # for creating regular densely-connected NN layer.
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout # for adding Concolutional and densely-connected NN layers.
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, Flatten,\
                                    Reshape, LeakyReLU as LR,\
                                    Activation, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version

import decimal
from decimal import Decimal
import os
import keras 
import tensorflow as tf
from tensorflow.keras.layers import Dropout,BatchNormalization,Conv2D,MaxPooling2D,Dense,Flatten
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers
from keras import callbacks
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.layers import Dense # for creating regular densely-connected NN layer.
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout,MaxPooling2D # for adding Concolutional and densely-connected NN layers.
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential

from pathlib import Path  

from sklearn.model_selection import train_test_split # for splitting the data into train and test samples
from sklearn.metrics import classification_report # for model evaluation metrics
from sklearn.preprocessing import OrdinalEncoder # for encoding labels
from keras.models import load_model
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix

from tensorflow.keras.utils import plot_model

from sklearn.utils.class_weight import compute_sample_weight

import pickle

pandas: 1.4.4
geopandas: 1.22.3
Tensorflow/Keras: 2.9.0
pandas: 1.4.4
numpy: 1.22.3


# functions

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.calibration import CalibrationDisplay
from sklearn.metrics import brier_score_loss

def plot_roc(pred,y):
    fpr, tpr, _ = roc_curve(y, pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()

In [3]:
def evaluate_perf(test_inputs,test_outputs,day=0):
    t = model.predict(test_inputs)
    one_day_t = t
    one_day_y = test_outputs
    
    ranges = [0.025,0.05,0.075,0.1]
    
    for j in ranges:
        classes = []
        for i in one_day_t.reshape(-1,1):
            if i<=j: classes.append(0)
            else: classes.append(1)
    
    # confusion matrix with threshold on 0.1, otherwise always 0 
        ConfusionMatrixDisplay(confusion_matrix(one_day_y.reshape(-1,1), classes)).plot(colorbar=False,cmap=plt.cm.Blues, values_format='d')
        ConfusionMatrixDisplay(confusion_matrix(one_day_y.reshape(-1,1), classes, normalize='true')).plot(colorbar=False,cmap=plt.cm.Blues)
    
    plot_roc(one_day_t.reshape(-1,1),one_day_y.reshape(-1,1))
    
    display = CalibrationDisplay.from_predictions(one_day_y.reshape(-1,1), one_day_t.reshape(-1,1), n_bins=10)

    print(f'All zeros Brier score: {brier_score_loss(one_day_y.reshape(-1,1), np.zeros(len(one_day_y.reshape(-1,1))))}')
    print(f'Model Brier score: {brier_score_loss(one_day_y.reshape(-1,1), one_day_t.reshape(-1,1))}')
    

# DataGenerator

In [7]:
from tensorflow import convert_to_tensor

class DataGenerator(Sequence):
    def __init__(self, x_paths, y_paths, batch_size=10512):
        self.x_paths = x_paths
        self.y_paths = y_paths
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x_paths) / self.batch_size))

    def __getitem__(self, idx):
        batch_x_paths = self.x_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y_paths = self.y_paths[idx * self.batch_size:(idx + 1) * self.batch_size]

        batch_x = [np.load(path) for path in batch_x_paths]
        batch_y = [np.load(path) for path in batch_y_paths]

        batch_x = [convert_to_tensor(x) for x in batch_x]
        batch_y = [convert_to_tensor(y) for y in batch_y]

        return np.array(batch_x), np.array(batch_y)

In [40]:
##################################training sets
folder_name = os.listdir('/Users/3rfanian/P_main/TC-main/Paolo/Notebooks_newTarget/imgs/train/imgs')
folder_name.remove('.DS_Store' )

path_training_img = '/Users/3rfanian/P_main/TC-main/Paolo/Notebooks_newTarget/imgs/train/imgs'
path_training_targets = '/Users/3rfanian/P_main/TC-main/Paolo/Notebooks_newTarget/imgs/train/targets'

path_list_img = []
path_list_targets = []

for fitem in folder_name:
    path_imgs = path_training_img+'/'+fitem+'/'
    path_list_img.append(path_imgs)

    path_targets = path_training_targets+'/'+fitem+'/'
    path_list_targets.append(path_targets)
    
trainin_imgs_addr = []
trainin_targets_addr = []

for item_path in path_list_img:
    folder_item = os.listdir(item_path)
    #print(type(str(folder_item)))
    paths_imgs = item_path + folder_item[0]
    trainin_imgs_addr.append(paths_imgs)

for item_path_t in path_list_targets:
    folder_item_t = os.listdir(item_path_t)
    #print(type(str(folder_item)))
    paths_targets = item_path_t + folder_item_t[0]
    trainin_targets_addr.append(paths_targets)
    
##################################validation sets
folder_name_val = os.listdir('/Users/3rfanian/P_main/TC-main/Paolo/Notebooks_newTarget/imgs/val/imgs')
#folder_name_val .remove('.DS_Store' )

path_training_img_val = '/Users/3rfanian/P_main/TC-main/Paolo/Notebooks_newTarget/imgs/val/imgs'
path_training_targets_val = '/Users/3rfanian/P_main/TC-main/Paolo/Notebooks_newTarget/imgs/val/targets'

path_list_img_val = []
path_list_targets_val = []

for fitem_val in folder_name_val:
    path_imgs_val = path_training_img_val+'/'+fitem_val+'/'
    path_list_img_val.append(path_imgs_val)

    path_targets_val = path_training_targets_val+'/'+fitem_val+'/'
    path_list_targets_val.append(path_targets_val)
    
trainin_imgs_addr_val = []
trainin_targets_addr_val = []

for item_path_val in path_list_img_val:
    folder_item_val = os.listdir(item_path_val)
    #print(type(str(folder_item)))
    paths_imgs_val = item_path_val + folder_item_val[0]
    trainin_imgs_addr_val.append(paths_imgs_val)

for item_path_t_val in path_list_targets_val:
    folder_item_t_val = os.listdir(item_path_t_val)
    #print(type(str(folder_item)))
    paths_targets_val = item_path_t_val + folder_item_t_val[0]
    trainin_targets_addr_val.append(paths_targets_val)

In [41]:
data_generator = DataGenerator(trainin_imgs_addr, trainin_targets_addr)
for x,y in data_generator:
    print(x,y)


/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_21838/548173134.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(batch_x), np.array(batch_y)


[array([[[[-8.68579434e-01,  9.06195285e-01, -1.85125977e-01, ...,
            5.96873201e-01, -1.29857102e+00, -2.13075717e-01],
          [-8.49505397e-01,  9.02204134e-01, -1.78417081e-01, ...,
            5.96873201e-01, -1.29502768e+00, -2.13075717e-01],
          [-8.23842203e-01,  8.98423257e-01, -1.73141698e-01, ...,
            5.96873201e-01, -1.29170138e+00, -2.13075717e-01],
          ...,
          [-9.01125534e-01,  9.18677988e-01, -2.13822113e-01, ...,
            5.96873201e-01, -1.31004543e+00, -2.13075717e-01],
          [-8.92821999e-01,  9.14478561e-01, -2.02852308e-01, ...,
            5.96873201e-01, -1.30610873e+00, -2.13075717e-01],
          [-8.82584884e-01,  9.10309343e-01, -1.93281287e-01, ...,
            5.96873201e-01, -1.30228370e+00, -2.13075717e-01]],

         [[-9.02100763e-01,  9.66465331e-01, -1.16058029e-01, ...,
            5.96873201e-01, -1.27288287e+00, -2.13075717e-01],
          [-9.41131320e-01,  9.69184245e-01, -1.09981107e-01, ...,
      

In [42]:
data_generator_val = DataGenerator(trainin_imgs_addr_val, trainin_targets_addr_val)
for x,y in data_generator_val:
    print(x,y)


[array([[[[-5.07050331e-01,  6.20933819e-02, -1.00281330e+00, ...,
            5.95361384e-01, -9.78943321e-01, -2.13051244e-01],
          [-4.71689951e-01,  8.05696680e-02, -1.03031108e+00, ...,
            5.95361384e-01, -9.86922331e-01, -2.13051244e-01],
          [-4.41111506e-01,  9.77939771e-02, -1.05765526e+00, ...,
            5.95361384e-01, -9.95129218e-01, -2.13051244e-01],
          ...,
          [-6.32069110e-01, -1.92270874e-03, -9.20081767e-01, ...,
            5.95361384e-01, -9.56665459e-01, -2.13051244e-01],
          [-5.88612768e-01,  2.09433225e-02, -9.47674379e-01, ...,
            5.95361384e-01, -9.63732227e-01, -2.13051244e-01],
          [-5.46351607e-01,  4.22659114e-02, -9.75254076e-01, ...,
            5.95361384e-01, -9.71189335e-01, -2.13051244e-01]],

         [[-7.08071919e-01,  4.43420875e-01, -1.13561638e+00, ...,
            5.95361384e-01, -7.21025676e-01, -2.13051244e-01],
          [-4.46301413e-01,  4.98656659e-01, -1.15306629e+00, ...,
      

/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_21838/548173134.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(batch_x), np.array(batch_y)


# basic model

In [43]:
model = Sequential()

model.add(layers.Input(shape=(73, 144, 9)))
model.add(layers.Conv2D(8, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), padding='same'))

#model.add(layers.Flatten())

#model.add(layers.Reshape((2,4,16)))

model.add(layers.Conv2D(8, (3,3), activation='relu', padding='same'))
model.add(layers.UpSampling2D((2,2)))

model.add(layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')) # sigmoidal activation to have probabilities

model.add(layers.Cropping2D(cropping=((0,1),(0,0))))

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=20, 
        verbose=1, mode='auto', restore_best_weights=True)


Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_101 (Conv2D)         (None, 73, 144, 8)        656       
                                                                 
 max_pooling2d_49 (MaxPoolin  (None, 37, 72, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_102 (Conv2D)         (None, 37, 72, 8)         584       
                                                                 
 up_sampling2d_23 (UpSamplin  (None, 74, 144, 8)       0         
 g2D)                                                            
                                                                 
 conv2d_103 (Conv2D)         (None, 74, 144, 1)        73        
                                                                 
 cropping2d_22 (Cropping2D)  (None, 73, 144, 1)      

In [44]:
model.fit(data_generator, validation_data=data_generator_val,
        callbacks=[monitor],epochs=100, steps_per_epoch=len(data_generator), validation_steps=len(data_generator_val))

/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_21838/548173134.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(batch_x), np.array(batch_y)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).